# TensorBoard(Tensorflow下的可视化)

## 1、TensorBoard是做什么的？

提供了模型可视化的功能。

## 2、TensorBoard的安装
在TensorFlow2.0中，TensorBoard是默认安装好的，所以，可以进入到你的logs目录下，直接根据以下命令启动：   
* tensorboard --logdir logs

logdir指的是日志目录，每次训练模型时，TensorBoard会在日志目录中创建一个子目录，在其中写入日志，TensorBoard的web应用正是通过日志来感知模型的训练状态，然后更新到网页端。

## 3、Fashion_mnist数据训练过程可视化


In [11]:
# 了解导入库的版本和名称
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

1.14.0
sys.version_info(major=3, minor=7, micro=0, releaselevel='final', serial=0)
matplotlib 2.2.3
numpy 1.18.2
pandas 1.0.4
sklearn 0.19.2
tensorflow 1.14.0
tensorflow.python.keras.api._v1.keras 2.2.4-tf


In [12]:
#加载数据集
import keras.datasets.fashion_mnist as fashion_mnist

(x_train_all,y_train_all),(x_test,y_test)=fashion_mnist.load_data()
x_valid,x_train=x_train_all[:5000],x_train_all[5000:]
y_valid,y_train=y_train_all[:5000],y_train_all[5000:]

In [13]:
print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [14]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaled=scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaled=scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [15]:
import tensorflow as tf
import tensorboard
import datetime
#构建一个全链接网络
model=keras.models.Sequential() #添加API以顺序的方式添加元素
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300,activation="relu"))
model.add(keras.layers.Dense(100,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax"))

# model = keras.Sequential([
# keras.layers.Flatten(input_shape=(28,28)),
# keras.layers.Dense(300,activation='relu'),
# keras.layers.Dense(100,activation='relu'),
# keras.layers.Dense(10,activation='softmax')
# ])
model.compile(loss="sparse_categorical_crossentropy",optimizer="sgd",metrics=["accuracy"])

In [16]:
log_dir="logs\\fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,write_images=True)  # 定义TensorBoard对象

TensorBoard中的参数解析：
* log_dir：保存TensorBoard要解析的日志文件的目录的路径。
* histogram_freq：频率（在epoch中），计算模型层的激活和权重直方图。如果设置为0，则不会计算直方图。必须为直方图可视化指定验证数据（或拆分）。
* write_graph：是否在TensorBoard中可视化图像。当write_graph设置为True时，日志文件可能会变得非常大。
* write_grads：是否在TensorBoard中可视化渐变直方图。 histogram_freq必须大于0。
* batch_size：用以直方图计算的传入神经元网络输入批的大小。
* write_images：是否在TensorBoard中编写模型权重以显示为图像。
* embeddings_freq：将保存所选嵌入层的频率（在epoch中）。如果设置为0，则不会计算嵌入。要在TensorBoard的嵌入选项卡中显示的数据必须作为embeddings_data传递。
* embeddings_layer_names：要关注的层名称列表。如果为None或空列表，则将监测所有嵌入层。
* embeddings_metadata：将层名称映射到文件名的字典，其中保存了此嵌入层的元数据。如果相同的元数据文件用于所有嵌入层，则可以传递字符串。
* embeddings_data：要嵌入在embeddings_layer_names指定的层的数据。Numpy数组（如果模型有单个输入）或Numpy数组列表（如果模型有多个输入）。
* update_freq：‘batch’或’epoch’或整数。使用’batch’时，在每个batch后将损失和指标写入TensorBoard。这同样适用’epoch’。如果使用整数，比方说1000，回调将会在每1000个样本后将指标和损失写入TensorBoard。请注意，过于频繁地写入TensorBoard会降低您的训练速度。可能引发的异常：
* ValueError：如果设置了histogram_freq且未提供验证数据。

In [17]:
history=model.fit(x_train_scaled,y_train,epochs=10,validation_data=(x_valid_scaled,y_valid),callbacks=[tensorboard_callback])

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 11s 208us/sample - loss: 0.5433 - acc: 0.8085 - val_loss: 0.4099 - val_acc: 0.8534
Epoch 2/10
55000/55000 [==============================] - 14s 254us/sample - loss: 0.3919 - acc: 0.8593 - val_loss: 0.3727 - val_acc: 0.8684
Epoch 3/10
55000/55000 [==============================] - 14s 260us/sample - loss: 0.3524 - acc: 0.8731 - val_loss: 0.3496 - val_acc: 0.8718
Epoch 4/10
55000/55000 [==============================] - 13s 240us/sample - loss: 0.3285 - acc: 0.8812 - val_loss: 0.3346 - val_acc: 0.8790
Epoch 5/10
55000/55000 [==============================] - 12s 211us/sample - loss: 0.3088 - acc: 0.8877 - val_loss: 0.3187 - val_acc: 0.8860
Epoch 6/10
55000/55000 [==============================] - 11s 208us/sample - loss: 0.2920 - acc: 0.8949 - val_loss: 0.3308 - val_acc: 0.8762
Epoch 7/10
55000/55000 [==============================] - 11s 208us/sample - loss: 0.2793 - acc: 0.8987 -

In [9]:
test_loss,test_accuracy = model.evaluate(x_test_scaled,y_test,verbose=2)
print('test accuracy: ',test_accuracy)

ValueError: Tensor Tensor("loss/mul:0", shape=(), dtype=float32) is not an element of this graph.

![](PPT\第2章Tensorflow基础\\幻灯片12.JPG)
![](PPT\第2章Tensorflow基础\\幻灯片13.JPG)
![](PPT\第2章Tensorflow基础\\幻灯片14.JPG)
![](PPT\第2章Tensorflow基础\\幻灯片15.JPG)

通过TensorBoard提供的图标，我们可以清楚的知道训练模型时loss和accuracy在每一个epoch中是怎么变化的，甚至，在网页菜单栏我们可以看到，TensorBoard提供了查看其他内容的功能：
* 在 scalars 下可以看到 accuracy，cross entropy，dropout，bias，weights 等的趋势。
* 在 images 和 audio 下可以看到输入的数据。
* 在 graphs 中可以看到模型的结构。
* 在 histogram 可以看到 activations，gradients 或者 weights 等变量的每一步的分布，越靠前面就是越新的步数的结果。
* distribution 和 histogram 是两种不同的形式，可以看到整体的状况。
* 在 embedding 中可以看到用 PCA 主成分分析方法将高维数据投影到 3D 空间后的数据的关系。

### 实验五：
请把鸢尾花模型的训练过程，用TensorBoard进行展示。

In [1]:
import cv2